In [1]:
%pip install -i https://pypi.tuna.tsinghua.edu.cn/simple huggingface_hub
%pip install -i https://pypi.tuna.tsinghua.edu.cn/simple datasets
%pip install -i https://pypi.tuna.tsinghua.edu.cn/simple trl
%pip install -i https://pypi.tuna.tsinghua.edu.cn/simple --upgrade torch transformers trl

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
ERROR: Operation cancelled by user
^C
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [2]:
import transformers
print(transformers.__version__)

C:\Users\asus\.conda\envs\advprompter\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.51.3


In [3]:
from huggingface_hub import HfApi, HfFolder
from huggingface_hub import login
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

print(torch.__version__)
print('import done')

2.6.0+cu126
import done


In [ ]:
# log in

# 替换为你的Hugging Face令牌
hf_token = ""

# 登录Hugging Face
login(token=hf_token)

# 验证令牌是否设置正确
api = HfApi()
token = HfFolder.get_token()
if token:
    print("Hugging Face API 令牌已设置")
else:
    print("Hugging Face API 令牌未设置，请检查环境变量")

Hugging Face API 令牌已设置


In [5]:
# 配置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
torch.cuda.empty_cache()

# 检查是否有可用的 CUDA 设备
if torch.cuda.is_available():
    # 获取当前设备
    device = torch.cuda.current_device()
    # 获取总显存
    total_memory = torch.cuda.get_device_properties(device).total_memory
    # 获取已分配的显存
    allocated_memory = torch.cuda.memory_allocated(device)
    # 计算剩余显存
    free_memory = total_memory - allocated_memory
    
    print(f"总显存: {total_memory / 1024**3:.2f} GB")
    print(f"已分配显存: {allocated_memory / 1024**3:.2f} GB")
    print(f"剩余显存: {free_memory / 1024**3:.2f} GB")
else:
    print("没有可用的 CUDA 设备。")

总显存: 11.99 GB
已分配显存: 0.00 GB
剩余显存: 11.99 GB


In [7]:
# 修改后的格式转换函数
def format_instruction_example(example):
    instruction = example["instruction"]
    input_text = example["input"]
    response = example["output"]
    
    if input_text and input_text.strip() != "":
        text = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{response}"
    else:
        text = f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
    
    # 编码文本
    encoded = tokenizer(text, truncation=True, max_length=512)
    
    # 明确添加"text"字段
    encoded["text"] = text
    
    # 标签通常与输入相同
    encoded["labels"] = encoded["input_ids"].copy()
    
    return encoded

In [8]:
# # 加载模型和分词器
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")
# model = model.to(device)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 加载数据集
dataset = load_dataset("BRZ911/Medical_consultation_data_SFT", split="train")

# 获取数据集的前10%
train_dataset = dataset.select(range(int(len(dataset) * 0.01)))

# 应用格式转换
formatted_dataset = train_dataset.map(format_instruction_example)

print('model and dataset load done')

C:\Users\asus\.conda\envs\advprompter\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\asus\.cache\huggingface\hub\models--Qwen--Qwen3-0.6B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP downlo

model and dataset load done


In [11]:
# 查看数据集的字段信息
print("数据集字段:", train_dataset.features.keys())

# 查看第一个样本的内容
if len(train_dataset) > 0:
    print("\n第一个样本内容:", train_dataset[0])

数据集字段: dict_keys(['instruction', 'input', 'output'])

第一个样本内容: {'instruction': '宝宝现在纯母乳喂养，不吃奶瓶怎么办？有什么妙招？小便黄想给他喝水，可他白开水就是不喝，奶粉也不吃。', 'input': '', 'output': '你可以用小勺来喂宝宝喝水或者纯果汁。如果宝宝小于六个月大就纯母乳喂养最好。母亲平常多喝些水。'}


In [12]:
# 对未训练的模型进行测试：

# 生成测试函数
def generate_text(prompt, max_length=200):
    """使用模型生成回答，限制最大长度为200 tokens"""
    # 编码输入
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 生成文本（使用贪婪解码以确保确定性）
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=min(len(inputs.input_ids[0]) + max_length, 2048),  # 总长度限制
            temperature=0.0,      # 确定性解码
            num_beams=1,          # 禁用束搜索
            do_sample=False       # 不使用采样
        )
    
    # 解码并提取生成的部分
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text[len(prompt):].strip()  # 返回生成的部分

# 对数据集的前10个样本进行测试
for i, example in enumerate(train_dataset):
    # 使用正确的字段名
    instruction = example["instruction"]
    input_text = example["input"]
    
    # 构建提示词
    if input_text and input_text.strip() != "":
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    
    # 生成回答
    generated_response = generate_text(prompt)
    
    # 打印结果
    print(f"\n样本 {i+1} 的问题:")
    print(instruction[:100] + ("..." if len(instruction) > 100 else ""))  # 截断显示
    
    print(f"\n生成的回答 (限制200 tokens):")
    print(generated_response[:200] + ("..." if len(generated_response) > 200 else ""))
    
    print("\n实际回答参考:")
    print(example["output"][:200] + ("..." if len(example["output"]) > 200 else ""))
    
    print("\n" + "-" * 80)


C:\Users\asus\.conda\envs\advprompter\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\asus\.conda\envs\advprompter\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\asus\.conda\envs\advprompter\Lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



样本 1 的问题:
宝宝现在纯母乳喂养，不吃奶瓶怎么办？有什么妙招？小便黄想给他喝水，可他白开水就是不喝，奶粉也不吃。

生成的回答 (限制200 tokens):
好的，宝宝现在纯母乳喂养，不吃奶瓶，小便黄，想给他喝水，但白开水不喝，奶粉也不吃。这可能有几种情况需要考虑。首先，纯母乳喂养的宝宝通常会自然地接受母乳，但有时可能会出现一些特殊情况，比如乳头不适、乳汁分泌不足、或者宝宝对奶瓶的适应问题。小便黄可能是因为宝宝的尿液中含有较多的糖分，这可能是因为乳汁中的糖分较高，或者宝宝的消化系统需要更多的水分来处理这些糖分。

对于宝宝不吃奶瓶的情况，可以尝试以下几...

实际回答参考:
你可以用小勺来喂宝宝喝水或者纯果汁。如果宝宝小于六个月大就纯母乳喂养最好。母亲平常多喝些水。

--------------------------------------------------------------------------------

样本 2 的问题:
肝功和两对半检查正常，但总胆红素和间接胆红素偏高，分别为22.2和17.3.请问这是什么原因呢？如果不去管它，会有什么后果呢？

生成的回答 (限制200 tokens):
好的，用户提到肝功和两对半检查正常，但总胆红素和间接胆红素偏高，分别为22.2和17.3。首先，我需要确认用户的问题是否正确。总胆红素和间接胆红素偏高通常与肝功能异常有关，但用户说两对半检查正常，这可能意味着肝功能没有问题。接下来，我需要考虑可能的原因。

总胆红素偏高可能由多种因素引起，比如溶血、胆道阻塞、肝硬化、胆道疾病等。间接胆红素偏高可能与溶血有关，因为溶血会导致胆红素生成增加。用户提到两...

实际回答参考:
总胆红素和间接胆红素偏高可能是由于药物影响或饮酒等原因引起的。此外，饮食也可能对胆红素水平产生影响。如果不处理这个问题，可能会导致进一步的健康问题。建议您咨询医生以获取更详细的诊断和治疗建议。此外，多补充水果和蔬菜也是一个好的饮食习惯。

--------------------------------------------------------------------------------

样本 3 的问题:
宝宝前段时间感冒了、去医院检查了开了一些药吃了、但是最近开始是一直咳嗽、很长时间了也不见好、特别害怕引

KeyboardInterrupt: 

In [13]:
import nanogcg
from nanogcg import run, GCGConfig
# 假设前面的代码块已经在Notebook中执行过

# 定义对话和目标
messages = [{"role": "user", "content": "Translate this sentence: "}]
target = "This is a test."

# 禁用前缀缓存以避免格式不兼容问题
config = GCGConfig(
    num_steps=50,
    search_width=128,
    topk=64,
    n_replace=2,
    use_prefix_cache=False  # 禁用前缀缓存
)

result = run(model, tokenizer, messages, target, config)
print(f"Best string: {result.best_string}")
print(f"Best loss: {result.best_loss}")

ModuleNotFoundError: No module named 'nanogcg'

In [11]:
# 获取优化后的字符串
optimized_string = result.best_string

# 构建完整提示
prompt = f"Translate this sentence: {optimized_string}"

# 将提示转换为模型输入
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# 使用generate方法生成回复
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,  # 控制生成的最大长度
        temperature=0.7,    # 控制生成的随机性
        top_p=0.9,          # 控制采样策略
        do_sample=True      # 启用采样
    )

# 解码生成的回复
response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

print("Generated response:", response)

Generated response: ain) )) )) ) ... (MediaMeshPro augmentedanasitet node.) onto ... (MediaMeshPro augmentedanasitet node.) onto ... (MediaMeshPro augmentedanasitet node.) onto ... (MediaMeshPro augmentedanasitet node.) onto ... (MediaMeshPro augmented


In [8]:
# 检查模型参数所在的设备
for param in model.parameters():
    if param.device.type == 'cuda':
        print("模型已成功加载到 GPU 上。")
        break
else:
    print("模型未加载到 GPU 上，仍在 CPU 上。")

模型已成功加载到 GPU 上。


In [ ]:
# 修改后的格式转换函数
def format_instruction_example(example):
    instruction = example["instruction"]
    input_text = example["input"]
    response = example["output"]
    
    if input_text and input_text.strip() != "":
        text = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{response}"
    else:
        text = f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
    
    # 编码文本
    encoded = tokenizer(text, truncation=True, max_length=512)
    
    # 明确添加"text"字段
    encoded["text"] = text
    
    # 标签通常与输入相同
    encoded["labels"] = encoded["input_ids"].copy()
    
    return encoded

# 重新应用格式转换
formatted_dataset = train_dataset.map(format_instruction_example)

In [24]:
from trl import setup_chat_format
# Set up the chat format with default 'chatml' format
model, tokenizer = setup_chat_format(model, tokenizer)

In [25]:
try:

    # training_args = SFTConfig(output_dir="/tmp")
    # 定义 SFT 训练配置
    training_args = SFTConfig(
    # 输出目录
    output_dir="/tmp",
    # 训练轮数，减少轮数可以降低训练成本
    num_train_epochs=3,
    # 每批处理的样本数量，根据 GPU 内存调整
    per_device_train_batch_size=8,
    # 梯度累积步数，增大该值可以模拟更大的批次大小
    gradient_accumulation_steps=4,
    # 学习率，合适的学习率有助于模型收敛且降低成本
    learning_rate=2e-5,
    # 权重衰减，防止过拟合
    weight_decay=0.01,
    # 预热步数，在训练初期缓慢增加学习率
    warmup_steps=500,
    # 优化器使用 AdamW
    optim="adamw_torch",
    # 每多少步保存一次模型
    save_steps=10_000,
    # 每多少步记录一次日志
    logging_steps=100,
    # 混合精度训练，使用 fp16 可以减少内存使用和训练时间
    fp16=True,
)
    trainer = SFTTrainer(
        model,
        train_dataset=formatted_dataset,
        args=training_args,
    )

    trainer.train()

except Exception as e:
    print(f"训练过程中出现错误: {e}")

Tokenizing train dataset:   0%|          | 0/816477 [00:00<?, ? examples/s]

训练过程中出现错误: 'text'


In [ ]:
# 保存模型和分词器
save_path = r'C:\Users\asus\Desktop\fine_tuned_model'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"模型和分词器已保存到 {save_path}")

模型和分词器已保存到 C:\Users\asus\Desktop\fine_tuned_model


In [15]:
import gc

# 检查当前GPU内存使用情况
def print_gpu_memory():
    """打印当前GPU内存使用情况"""
    if torch.cuda.is_available():
        print(f"GPU内存使用: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
        print(f"GPU缓存使用: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")
    else:
        print("未检测到GPU")

# 打印卸载前的GPU内存状态
print("卸载前的GPU状态:")
print_gpu_memory()

# 卸载模型和分词器
try:
    # 将模型移回CPU
    if 'model' in locals() or 'model' in globals():
        model.to('cpu')
        del model
        print("✅ 模型已移至CPU并删除")
    else:
        print("⚠️ 未找到模型变量")
    
    # 删除分词器（如果需要释放更多内存）
    if 'tokenizer' in locals() or 'tokenizer' in globals():
        del tokenizer
        print("✅ 分词器已删除")
    else:
        print("⚠️ 未找到分词器变量")
    
    # 删除其他可能占用GPU内存的变量
    if 'train_dataset' in locals() or 'train_dataset' in globals():
        del train_dataset
    if 'val_dataset' in locals() or 'val_dataset' in globals():
        del val_dataset
    
    # 清空PyTorch缓存
    torch.cuda.empty_cache()
    
    # 强制垃圾回收
    gc.collect()
    
    # 打印卸载后的GPU内存状态
    print("\n卸载后的GPU状态:")
    print_gpu_memory()

except Exception as e:
    print(f"卸载过程中出错: {e}")

卸载前的GPU状态:
GPU内存使用: 2274.25 MB
GPU缓存使用: 2276.00 MB
✅ 模型已移至CPU并删除
✅ 分词器已删除

卸载后的GPU状态:
GPU内存使用: 0.00 MB
GPU缓存使用: 0.00 MB


In [ ]:
# 加载微调过的模型和分词器
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
    low_cpu_mem_usage=True
).to(device)

# 设置流式输出
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# 准备输入
prompt = "今天天气很好。"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 生成参数
generate_config = {
    "max_length": 100,
    "temperature": 0.7,
    "top_p": 0.9,
    "do_sample": True,
}

# 生成文本
with torch.no_grad():
    outputs = model.generate(**inputs, streamer=streamer, **generate_config)

# 解码输出（如果没有使用streamer）
if not streamer:
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)

In [ ]:
# 目标文本（需精确匹配）
target_start = "根据您描述的症状，可能是上呼吸道感染引起的。"
target_start_ids = tokenizer(target_start, return_tensors="pt").input_ids.squeeze().tolist()

# 超参数
n_tokens = 15          # 初始提示词长度调长（原10过短）
num_steps = 150         # 增加GCG迭代次数
k = 128                # 扩大top-k候选范围，增加多样性
B = 64                 # 增加候选提示词数量
sample_size = n_tokens      # 梯度计算采样token数（仅用于可视化，实际用全梯度）
max_generation_length = len(target_start_ids) + 20  # 限制生成长度

In [ ]:
def compute_loss(logits, target_start_ids):
    """严格匹配整个目标文本的损失函数"""
    target_tensor = torch.tensor(target_start_ids, device=logits.device).long()
    batch_size, seq_len, vocab_size = logits.shape
    target_len = len(target_start_ids)
    valid_positions = seq_len - target_len + 1  # 有效匹配位置数
    
    if valid_positions <= 0:
        return torch.tensor(0.0, device=logits.device)  # 避免除零
    
    total_loss = 0.0
    for i in range(batch_size):
        # 仅计算完全匹配目标长度的子序列损失
        if seq_len >= target_len:
            sub_logits = logits[i, :target_len]  # 强制匹配前target_len个token
            loss = F.cross_entropy(sub_logits.view(-1, vocab_size), target_tensor)
            total_loss += loss
    
    return total_loss / (batch_size * max(1, valid_positions))  # 防止除零

In [ ]:
def calculate_gradient(model, input_ids, position, vocab_size, target_start_ids):
    """
    计算单个位置的token梯度（one-hot编码）
    :return: 梯度向量（形状[vocab_size]，值越小表示损失越低）
    """
    embed_layer = model.get_input_embeddings()
    original_token = input_ids[position].item()
    vocab_size_model = embed_layer.weight.size(0)  # 获取模型嵌入矩阵的输入维度

    # 生成one-hot编码并替换嵌入
    one_hot = torch.zeros(vocab_size_model, device=device)
    one_hot[original_token] = 1.0
    one_hot.requires_grad_()

    input_embeds = embed_layer(input_ids.unsqueeze(0))  # 原始嵌入[1, seq_len, embed_dim]
    input_embeds[0, position] = one_hot @ embed_layer.weight  # 替换当前位置嵌入

    # 前向传播+损失计算+反向传播
    with torch.enable_grad():
        logits = model(inputs_embeds=input_embeds).logits
        loss = compute_loss(logits, target_start_ids)
        loss.backward()

    return one_hot.grad  # 返回每个token的梯度

In [ ]:
def gcg_algorithm():
    # 初始化提示词：随机生成n_tokens长度的token序列
    vocab_size = tokenizer.vocab_size
    initial_prompt = torch.randint(0, vocab_size, (n_tokens,), device=device).tolist()
    current_prompt = initial_prompt.copy()
    step_losses = []
    
    print(f"[INIT] Initial Prompt: {tokenizer.decode(current_prompt)}")
    
    for step in range(num_steps):
        # 转换为tensor
        input_ids = torch.tensor(current_prompt, device=device).long()
        grads = []  # 存储每个位置的梯度（形状[seq_len, vocab_size]）
        
        # 计算每个位置的梯度
        for pos in range(n_tokens):
            grad = calculate_gradient(model, input_ids, pos, vocab_size, target_start_ids)
            grads.append(grad.cpu().numpy())  # 转换为CPU数组
        
        # 生成候选提示（梯度引导）
        candidates = []
        for _ in range(B):
            new_prompt = current_prompt.copy()
            pos = np.random.randint(n_tokens)  # 随机选择修改位置
            
            # 选择梯度最小的k个token（梯度越小，损失下降潜力越大）
            top_k_indices = np.argsort(grads[pos])[:k]  # 按梯度升序排列
            new_token = np.random.choice(top_k_indices)  # 随机选择一个候选token
            new_prompt[pos] = new_token
            candidates.append(new_prompt)
        
        # 评估候选损失
        best_loss = float('inf')
        best_candidate = current_prompt
        for cand in candidates:
            input_ids_cand = torch.tensor(cand, device=device).long().unsqueeze(0)
            with torch.no_grad():
                logits = model(input_ids_cand).logits
                loss = compute_loss(logits, target_start_ids)
            
            if loss.item() < best_loss:
                best_loss = loss.item()
                best_candidate = cand
        
        # 更新提示词
        current_prompt = best_candidate
        step_losses.append(best_loss)
        print(f"[STEP {step + 1}] Loss: {best_loss:.4f}, Prompt: {tokenizer.decode(current_prompt)}")
    
    # 绘制损失曲线
    plt.figure(figsize=(12, 6))
    plt.plot(step_losses, label="Loss")
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.title("GCG Optimization Loss Curve")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return current_prompt

In [ ]:
def miniprompt_algorithm():
    global n_tokens
    global num_steps
    running_min = 0
    running_max = 100000
    best = None
    while True:
        z = gcg_algorithm()
        input_ids = torch.tensor(z, device=device).long().unsqueeze(0)
        attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)
        
        # 设置填充标记ID，如果模型没有指定的话
        if tokenizer.pad_token_id is None:
            tokenizer.pad_token_id = tokenizer.eos_token_id
        
        with torch.no_grad():
            # 显式要求生成目标文本长度，避免截断
            # generated_output = model.generate(
            #     input_ids,
            #     attention_mask=attention_mask,
            #     pad_token_id=tokenizer.eos_token_id,
            #     max_length=len(target_start_ids) + 20,  # 增加冗余长度
            #     min_length=len(target_start_ids),       # 最小生成长度等于目标长度
            #     eos_token_id=tokenizer.eos_token_id,
            #     early_stopping=False
            # )
            generated_output = model.generate(
                input_ids,
                attention_mask=attention_mask,
                pad_token_id=tokenizer.pad_token_id,
                num_beams=50,
                no_repeat_ngram_size=2,
                early_stopping=False,  # 不提前停止
                temperature=0.2
            )
            generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
            print(f"Generated text: {generated_text} :end")
            print(f"with token of {n_tokens}")
            
            # 严格检查是否完全匹配（包括大小写和标点）
            if generated_text.strip() == target_start.strip():
                running_max = n_tokens
                n_tokens = max(1, n_tokens - 1)  # 至少保留1个token
                best = z
                num_steps = max(50, num_steps - 10)  # 避免迭代次数过低
                print(f"✅ Matched at {n_tokens} tokens!")
            else:
                running_min = n_tokens
                n_tokens = min(n_tokens + 5, running_max)  # 每次增加5个token，加速搜索
                num_steps = min(500, num_steps + 50)       # 增加迭代次数上限
                print(f"❌ Not matched, increase to {n_tokens} tokens")

    return best

In [ ]:
# test for miniprompt

final_prompt = miniprompt_algorithm()

# 计算ACR
if final_prompt:
    target_length = len(target_start_ids)
    min_prompt_length = len(final_prompt)
    acr = target_length / min_prompt_length if min_prompt_length > 0 else 0
else:
    acr = 0

# 输出最终结果
print("\n" + "=" * 50)
if final_prompt:
    print(f"Final Optimized Prompt (Tokens: {len(final_prompt)}):")
    print(f"  Text: {tokenizer.decode(final_prompt)}")
    print(f"  Tokens: {final_prompt}")
else:
    print("No valid prompt found.")
print(f"ACR: {acr}")
print("=" * 50)

In [ ]:
model.eval()

while True:
    # 获取用户输入
    user_input = input("你: ")
    if user_input == "退出":
        break

    # 对用户输入进行编码
    input_ids = tokenizer.encode(user_input, return_tensors='pt').to(device)
    # 创建注意力掩码
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)

    # 设置填充标记ID，如果模型没有指定的话
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
        
    temp = float(0.8)

    # 生成模型回复
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        pad_token_id=tokenizer.pad_token_id,
        num_beams=15,
        no_repeat_ngram_size=2,
        early_stopping=False,  # 不提前停止
        temperature=temp,
        do_sample=True
    )

    # 对模型输出进行解码
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # 统计输出的token数
    tokens = tokenizer.tokenize(response)
    token_count = len(tokens)

    # 提取模型回复（去除用户输入部分）
    response = response[len(user_input):].strip()

    print(f"模型: {response}")
    print(f"输出的token数: {token_count}")

print("对话结束。")